In [97]:
MYSQL_CONFIG = {
    "USER_NAME": "intern_test",
    "PASSWORD": "intern_test",
    "HOST": "192.168.80.34",
    "PORT": "2023",
    "DATABASE": "postgres"
}

DATA_BASE_DESIGN = "https://dbdiagram.io/d/6526d848ffbf5169f080dc9e"

In [101]:
from sqlalchemy import Column, Enum, Integer, String, DateTime, ForeignKey, Float
from sqlalchemy.ext.declarative import declarative_base
import datetime
import enum


Base = declarative_base()
schema_student = {"schema": "student"}
# schema = {"schema": "ercot"}
expires_at_default = datetime.datetime.utcnow() + datetime.timedelta(days=90)

class GenderEnum(enum.Enum):
    Male = "Male"
    Female = "Female"


class Student(Base):
    __tablename__ = 'student'
    # __table_args__ = schema_student

    student_id = Column(String, primary_key=True)
    student_name = Column(String)
    created_datetime = Column(DateTime, default=datetime.datetime.utcnow())
    last_modified_datetime = Column(DateTime, default=datetime.datetime.utcnow())
    expires_at = Column(DateTime, default=expires_at_default)


class Country(Base):
    __tablename__ = 'country'
    # __table_args__ = schema_student

    country_id = Column(Integer, primary_key=True)
    country_name = Column(String)
    area = Column(String)
    population = Column(Integer)
    acreage = Column(Float)
    density = Column(Integer)
    administration_district = Column(Integer)


class StudentAuthorization(Base):
    __tablename__ = 'student_authorization'
    # __table_args__ = schema_student

    user_authorization_id = Column(String, primary_key=True)
    student_id = Column(String, ForeignKey('student.student_id'))
    country_id = Column(Integer, ForeignKey('country.country_id'))
    created_datetime = Column(DateTime, default=datetime.datetime.utcnow())


class StudentInformation(Base):
    __tablename__ = 'student_information'
    # __table_args__ = schema_student

    student_id = Column(String, ForeignKey('student.student_id'), primary_key=True)
    country_id = Column(Integer, ForeignKey('country.country_id'), primary_key=True)
    email = Column(String)
    gender = Column(Enum(GenderEnum))
    date_of_birth = Column(String, default=datetime.datetime.utcnow().strftime("%Y-%m-%d"))


class StudentScore(Base):
    __tablename__ = 'student_score'
    # __table_args__ = schema_student

    user_authorization_id = Column(String, ForeignKey('student_authorization.user_authorization_id'), primary_key=True)
    math = Column(Float)
    literature = Column(Float)
    english = Column(Float)
    physics = Column(Float)
    chemistry = Column(Float)

C:\Users\Admin\AppData\Local\Temp\ipykernel_20732\113529374.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [102]:
from sqlalchemy.orm import sessionmaker, Session
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine

url_object = URL.create(
                    drivername="postgresql",
                    username=MYSQL_CONFIG["USER_NAME"],
                    password=MYSQL_CONFIG["PASSWORD"],
                    host=MYSQL_CONFIG["HOST"],
                    port=MYSQL_CONFIG["PORT"],
                    database=MYSQL_CONFIG["DATABASE"],
                )
engine = create_engine(url_object, echo=True)
Session_ = sessionmaker(bind=engine)
session: Session = Session_()

session.query(Country).filter(Country.country_id == 1).all() # SELECT * FROM adress WHERE address_id = 1;

2023-10-24 17:58:56,067 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-10-24 17:58:56,067 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-24 17:58:56,098 INFO sqlalchemy.engine.Engine select current_schema()
2023-10-24 17:58:56,100 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-24 17:58:56,136 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-10-24 17:58:56,136 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-24 17:58:56,173 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-24 17:58:56,193 INFO sqlalchemy.engine.Engine SELECT country.country_id AS country_country_id, country.country_name AS country_country_name, country.area AS country_area, country.population AS country_population, country.acreage AS country_acreage, country.density AS country_density, country.administration_district AS country_administration_district 
FROM country 
WHERE country.country_id = %(country_id_1)s
2023-10-24 17:58:56,195 INFO sqlalchemy.engine.Engine [generat

In [67]:
data = session.query(StudentScore).all()
ans = 0
for i in data:
    sum = i.math + i.literature + i.english + i.physics + i.chemistry
    sum_id = i.user_authorization_id
    if sum > ans:
        ans = sum
        ans_id = sum_id
        
print(ans_id)

ans_id = session.query(StudentScore).all()

2023-10-24 17:35:23,821 INFO sqlalchemy.engine.Engine SELECT student_score.user_authorization_id AS student_score_user_authorization_id, student_score.math AS student_score_math, student_score.literature AS student_score_literature, student_score.english AS student_score_english, student_score.physics AS student_score_physics, student_score.chemistry AS student_score_chemistry 
FROM student_score
2023-10-24 17:35:23,823 INFO sqlalchemy.engine.Engine [generated in 0.00205s] {}
f2ed7a18-2b03-4174-83a9-95d27fb8eb7e
2023-10-24 17:35:33,922 INFO sqlalchemy.engine.Engine SELECT student_score.user_authorization_id AS student_score_user_authorization_id, student_score.math AS student_score_math, student_score.literature AS student_score_literature, student_score.english AS student_score_english, student_score.physics AS student_score_physics, student_score.chemistry AS student_score_chemistry 
FROM student_score
2023-10-24 17:35:33,922 INFO sqlalchemy.engine.Engine [cached since 10.1s ago] {}


In [68]:
data = session.query(StudentAuthorization).filter(StudentAuthorization.country_id == 24).all()
ans_id = None
ans = 0
for i in data:
    student = session.query(StudentScore).filter(StudentScore.user_authorization_id == i.user_authorization_id).all()
    sum = student[0].math + student[0].literature + student[0].english + student[0].physics + student[0].chemistry
    sum_id = student[0].user_authorization_id
    if sum > ans:
        ans = sum
        ans_id = sum_id
        
print(ans_id) 
data = session.query(StudentAuthorization).filter(StudentAuthorization.user_authorization_id == ans_id).all()
print(data.student_id)

2023-10-24 17:35:39,292 INFO sqlalchemy.engine.Engine SELECT student_authorization.user_authorization_id AS student_authorization_user_authorization_id, student_authorization.student_id AS student_authorization_student_id, student_authorization.country_id AS student_authorization_country_id, student_authorization.created_datetime AS student_authorization_created_datetime 
FROM student_authorization 
WHERE student_authorization.country_id = %(country_id_1)s
2023-10-24 17:35:39,294 INFO sqlalchemy.engine.Engine [generated in 0.00189s] {'country_id_1': 24}
2023-10-24 17:35:39,955 INFO sqlalchemy.engine.Engine SELECT student_score.user_authorization_id AS student_score_user_authorization_id, student_score.math AS student_score_math, student_score.literature AS student_score_literature, student_score.english AS student_score_english, student_score.physics AS student_score_physics, student_score.chemistry AS student_score_chemistry 
FROM student_score 
WHERE student_score.user_authorization_

AttributeError: 'list' object has no attribute 'student_id'

In [69]:
print(data[0].student_id)

20190597


In [70]:
data = session.query(StudentAuthorization).filter(StudentAuthorization.country_id == 41).all()
count = 0
ans_id = None
ans = 0
for i in data:
    student = session.query(StudentScore).filter(StudentScore.user_authorization_id == i.user_authorization_id).all()
    if student[0].physics > 9.6:
        count += 1
print(count)

2023-10-24 17:36:07,001 INFO sqlalchemy.engine.Engine SELECT student_authorization.user_authorization_id AS student_authorization_user_authorization_id, student_authorization.student_id AS student_authorization_student_id, student_authorization.country_id AS student_authorization_country_id, student_authorization.created_datetime AS student_authorization_created_datetime 
FROM student_authorization 
WHERE student_authorization.country_id = %(country_id_1)s
2023-10-24 17:36:07,003 INFO sqlalchemy.engine.Engine [cached since 27.71s ago] {'country_id_1': 41}
2023-10-24 17:36:07,156 INFO sqlalchemy.engine.Engine SELECT student_score.user_authorization_id AS student_score_user_authorization_id, student_score.math AS student_score_math, student_score.literature AS student_score_literature, student_score.english AS student_score_english, student_score.physics AS student_score_physics, student_score.chemistry AS student_score_chemistry 
FROM student_score 
WHERE student_score.user_authorizatio

In [71]:
from sqlalchemy import create_engine, func

# Query the number of students for each math score
result = session.query(StudentScore.math, func.count(StudentScore.user_authorization_id)).group_by(StudentScore.math).all()

# Iterate over the result and print the math score and number of students
for score, count in result:
    print(f"Math Score: {score}, Number of Students: {count}")

2023-10-24 17:37:53,365 INFO sqlalchemy.engine.Engine SELECT student_score.math AS student_score_math, count(student_score.user_authorization_id) AS count_1 
FROM student_score GROUP BY student_score.math
2023-10-24 17:37:53,366 INFO sqlalchemy.engine.Engine [generated in 0.00128s] {}
Math Score: 9.8, Number of Students: 15098
Math Score: 8.8, Number of Students: 14866
Math Score: 9.2, Number of Students: 15087
Math Score: 9.0, Number of Students: 14877
Math Score: 9.6, Number of Students: 15038
Math Score: 9.4, Number of Students: 15034


In [111]:
import functools
data = session.query(StudentScore).all()
class student_copy:
    user_id = None
    sum_score = None
    sum_4 = None
    sum_5 = None
def compare_obj(s1, s2):
    if s1.sum_score > s2.sum_score:
        return -1
    elif s1.sum_score < s2.sum_score:
        return 1
    else:
        if s1.sum_4 > s2.sum_4:
            return -1
        elif s1.sum_4 < s2.sum_4:
            return 1
        elif s1.sum_5 > s2.sum_5:
            return -1
        elif s1.sum_5 < s2.sum_5:
            return 1
        else:
            return 0  
students = []   
for i in data:
    s = student_copy()
    sum_score_1 = i.math * 2 + i.physics + i.english
    sum_score_2 = i.math * 2 + i.physics + i.chemistry
    if sum_score_1 > sum_score_2:
        s.sum_score = sum_score_1
    else:
        s.sum_score = sum_score_2
    s.sum_4 = i.math + i.physics + i.english + i.chemistry
    s.sum_5 = s.sum_4 + i.literature
    s.user_id = i.user_authorization_id
    students.append(s)
    # print(s.sum_score)
    
sorted_objects = sorted(students, key=functools.cmp_to_key(compare_obj))
students = sorted_objects[:1200]
check = False
for i in range(len(sorted_objects)):
    if i >= 1200:
        if(sorted_objects[i].sum_5 == sorted_objects[1199].sum_5):
            print(i)
            students.append(sorted_objects[i])
        else:
            break

c1 = 0
c2 = 0
for i in students:
    if i.sum_score == 39.2:
        c1 += 1
    else:
        c2 += 1
print(len(students))
print(c1)
print(c2)



2023-10-24 18:03:31,682 INFO sqlalchemy.engine.Engine SELECT student_score.user_authorization_id AS student_score_user_authorization_id, student_score.math AS student_score_math, student_score.literature AS student_score_literature, student_score.english AS student_score_english, student_score.physics AS student_score_physics, student_score.chemistry AS student_score_chemistry 
FROM student_score
2023-10-24 18:03:31,684 INFO sqlalchemy.engine.Engine [cached since 261.4s ago] {}
1200
1201
1202
1203
1204
1205
1206
1207
1208
777
431


In [110]:
if(sorted_objects[1203].sum_5 == sorted_objects[1199].sum_5):
    print(1)

1
